# Лабораторна робота №3

Фатенко Владислав, КА-61м, Варіант 16

## Завдання

Виконати ранжування альтернатив методом PROMETHEE II для критеріїв c2, c3, c7.

Розглядається задача №9. Матриця рішень, тобто оцінки альтернатив за критеріями, наступні:

|        | $c_2$ | $c_3$ | $c_7$ |
|:------:|:-----:|-------|-------|
|  $a_1$ |  922  | 7.22  | 318   |
|  $a_2$ |  900  | 11.45 | 303   |
| $a_3$  |  883  | 9.86  | 311   |
| $a_4$  |  840  | 10.38 | 318   |
| $a_5$  | 903   | 10.74 | 281   |
| $a_6$  | 922   | 13.87 | 269   |
| $a_7$  | 769   | 9.33  | 180   |
| $a_8$  | 896   | 9.82  | 121   |
| $a_9$  | 770   | 9.39  | 228   |
| $a_{10}$ | 766   | 7.22  | 157   |
| $a_{11}$ | 897   | 10.61 | 162   |

Ваги критеріїв $W$, пороги нерозрізненості $Q$ і переваги $P$, дисперсії $S$ наступні:

|       | $c_2$ | $c_3$ | $c_7$ |
|:-----:|:-----:|:-----:|:-----:|
| $w^C$ |0.033 |0.033|0.097 |
|  $Q$  |  30  |  1  |  30  |
|  $P$  |  60  |  2  |  60  |
|  $S$  |  -   |  1  |  -   |
|  Тип  |linear|gauss|level|

Критерії $c_2,c_7$ - критерії доходів, а $c_3$ - критерій витрат.

## Виконання роботи

### Імпорт бібліотек

In [1]:
%pylab inline
import seaborn as sns
import scipy as sp
import pandas as pd
from scipy import stats
from IPython.display import display, Latex

Populating the interactive namespace from numpy and matplotlib


### Підготовка програмних функцій

Створимо функції переваги кожного з розглянутих у лекції типів:
- usual shape: $H(f) = [f > 0]$
- U-shape: $H(f\;|\;p)=[f > p]$
- level-shape: $H(f\;|\;p,q)=([f>q]+[f>p])/2$
- V-shape: $H(f\;|\;p) = \frac{\min(f,p)}{p}[f>0]$
- linear shape: $H(f\;|\;p,q)=\frac{\min(f-q,p-q)}{p-q}[f>q]$
- gaussian shape: $H(f\;|\;s) = (1 - \exp\left(-\frac{f^2}{2s^2}\right)[f > 0]$

In [2]:
def usual_shape():
    def func(f):
        return float64(f > 0)
    return func

def u_shape(p):
    assert p > 0, "p <= 0"
    def func(f):
        return float64(f > p)
    return func
    
def level_shape(p, q):
    assert p > 0, "p <= 0"
    assert q > 0, "q <= 0"
    def func(f):
        return (float64(f > q) + (f > p)) / 2
    return func
    
def v_shape(p):
    assert p > 0, "p <= 0"
    def func(f):
        return minimum(f, p) * (f > 0) / p
    return func

def linear_shape(p, q):
    assert p > 0, "p <= 0"
    assert q > 0, "q <= 0"
    assert p > q, "p <= q"
    p -= q
    def func(f):
        return minimum(f - q, p) * (f > q) / p
    return func

def gauss_shape(s):
    assert s > 0, "s <= 0"
    def func(f):
        return (1 - exp(- f**2 / (2 * s**2))) * (f > 0)
    return func

Створимо функцию генерації МПП з вихідних даних:

In [3]:
def generate_pcm(*marks):
    num_alt = marks[0].shape[0]
    pcm = zeros((len(marks), num_alt, num_alt))
    for i, mark in enumerate(marks):
        pcm[i] = mark[:, newaxis] - mark
    return pcm

Створимо функцію, що буде створювати агрегатор ступеней порівнянь по заданим критеріям і вагам:

In [4]:
def aggr_adv_deg(weights, *funcs):
    assert weights.shape[0] == len(funcs), "weights len does not correspond to number of criterions"
    def aggregator(arr):
        assert arr.shape[0] == len(funcs), "array shape does not correspond to number of criterions"
        res = zeros(arr.shape)
        for i, func in enumerate(funcs):
            res[i] = func(arr[i])
        return average(res, axis=0, weights=weights)
    return aggregator

### Введення даних

In [5]:
c2_marks = array([922, 900, 883, 840, 903, 922, 769, 896, 770, 766, 897], dtype='float64')
c3_marks = array([7.22, 11.45, 9.86, 10.38, 10.74, 13.87, 9.33, 9.82, 9.39, 7.22, 10.61], dtype='float64')
c7_marks = array([318, 303, 311, 318, 281, 269, 180, 121, 228, 157, 162], dtype='float64')
weights_crit = array([0.033, 0.033, 0.097], dtype='float64')
crit1 = linear_shape(60, 30)
crit2 = gauss_shape(1)
crit3 = level_shape(60, 30)

Оскільки $c_3$ – показник витрат, то його можна звести до показника доходу, помноживши на $-1$:

In [6]:
c3_marks_adj = -c3_marks

Створимо матрицю порівнянь за кожним з розглянутих критеріїв:

In [7]:
pcm = generate_pcm(c2_marks, c3_marks_adj, c7_marks)
names = ['$c_2$','$c_3^*$','$c_7$']
alternatives = ['$a_{%d}$' % (i + 1) for i in range(len(c2_marks))]
for i, name in enumerate(names):
    display(Latex('Матриця порівнянь для критерію %s:' % name))
    display(pd.DataFrame(pcm[i], columns=alternatives, index=alternatives))

<IPython.core.display.Latex object>

,$a_{1}$,$a_{2}$,$a_{3}$,$a_{4}$,$a_{5}$,$a_{6}$,$a_{7}$,$a_{8}$,$a_{9}$,$a_{10}$,$a_{11}$
$a_{1}$,0.0,22.0,39.0,82.0,19.0,0.0,153.0,26.0,152.0,156.0,25.0
$a_{2}$,-22.0,0.0,17.0,60.0,-3.0,-22.0,131.0,4.0,130.0,134.0,3.0
$a_{3}$,-39.0,-17.0,0.0,43.0,-20.0,-39.0,114.0,-13.0,113.0,117.0,-14.0
$a_{4}$,-82.0,-60.0,-43.0,0.0,-63.0,-82.0,71.0,-56.0,70.0,74.0,-57.0
$a_{5}$,-19.0,3.0,20.0,63.0,0.0,-19.0,134.0,7.0,133.0,137.0,6.0
$a_{6}$,0.0,22.0,39.0,82.0,19.0,0.0,153.0,26.0,152.0,156.0,25.0
$a_{7}$,-153.0,-131.0,-114.0,-71.0,-134.0,-153.0,0.0,-127.0,-1.0,3.0,-128.0
$a_{8}$,-26.0,-4.0,13.0,56.0,-7.0,-26.0,127.0,0.0,126.0,130.0,-1.0
$a_{9}$,-152.0,-130.0,-113.0,-70.0,-133.0,-152.0,1.0,-126.0,0.0,4.0,-127.0
$a_{10}$,-156.0,-134.0,-117.0,-74.0,-137.0,-156.0,-3.0,-130.0,-4.0,0.0,-131.0


<IPython.core.display.Latex object>

,$a_{1}$,$a_{2}$,$a_{3}$,$a_{4}$,$a_{5}$,$a_{6}$,$a_{7}$,$a_{8}$,$a_{9}$,$a_{10}$,$a_{11}$
$a_{1}$,0.00,4.23,2.64,3.16,3.52,6.65,2.11,2.60,2.17,0.00,3.39
$a_{2}$,-4.23,0.00,-1.59,-1.07,-0.71,2.42,-2.12,-1.63,-2.06,-4.23,-0.84
$a_{3}$,-2.64,1.59,0.00,0.52,0.88,4.01,-0.53,-0.04,-0.47,-2.64,0.75
$a_{4}$,-3.16,1.07,-0.52,0.00,0.36,3.49,-1.05,-0.56,-0.99,-3.16,0.23
$a_{5}$,-3.52,0.71,-0.88,-0.36,0.00,3.13,-1.41,-0.92,-1.35,-3.52,-0.13
$a_{6}$,-6.65,-2.42,-4.01,-3.49,-3.13,0.00,-4.54,-4.05,-4.48,-6.65,-3.26
$a_{7}$,-2.11,2.12,0.53,1.05,1.41,4.54,0.00,0.49,0.06,-2.11,1.28
$a_{8}$,-2.60,1.63,0.04,0.56,0.92,4.05,-0.49,0.00,-0.43,-2.60,0.79
$a_{9}$,-2.17,2.06,0.47,0.99,1.35,4.48,-0.06,0.43,0.00,-2.17,1.22
$a_{10}$,0.00,4.23,2.64,3.16,3.52,6.65,2.11,2.60,2.17,0.00,3.39


<IPython.core.display.Latex object>

,$a_{1}$,$a_{2}$,$a_{3}$,$a_{4}$,$a_{5}$,$a_{6}$,$a_{7}$,$a_{8}$,$a_{9}$,$a_{10}$,$a_{11}$
$a_{1}$,0.0,15.0,7.0,0.0,37.0,49.0,138.0,197.0,90.0,161.0,156.0
$a_{2}$,-15.0,0.0,-8.0,-15.0,22.0,34.0,123.0,182.0,75.0,146.0,141.0
$a_{3}$,-7.0,8.0,0.0,-7.0,30.0,42.0,131.0,190.0,83.0,154.0,149.0
$a_{4}$,0.0,15.0,7.0,0.0,37.0,49.0,138.0,197.0,90.0,161.0,156.0
$a_{5}$,-37.0,-22.0,-30.0,-37.0,0.0,12.0,101.0,160.0,53.0,124.0,119.0
$a_{6}$,-49.0,-34.0,-42.0,-49.0,-12.0,0.0,89.0,148.0,41.0,112.0,107.0
$a_{7}$,-138.0,-123.0,-131.0,-138.0,-101.0,-89.0,0.0,59.0,-48.0,23.0,18.0
$a_{8}$,-197.0,-182.0,-190.0,-197.0,-160.0,-148.0,-59.0,0.0,-107.0,-36.0,-41.0
$a_{9}$,-90.0,-75.0,-83.0,-90.0,-53.0,-41.0,48.0,107.0,0.0,71.0,66.0
$a_{10}$,-161.0,-146.0,-154.0,-161.0,-124.0,-112.0,-23.0,36.0,-71.0,0.0,-5.0


Побудуємо агрегатор з заданими функціями переваг:

In [8]:
aggregator = aggr_adv_deg(weights_crit, crit1, crit2, crit3)

Отримали наступну агреговану матрицю попарних порівнянь:

In [9]:
agg_pcm = aggregator(pcm)
display(pd.DataFrame(agg_pcm.round(3), columns=alternatives, index=alternatives))

,$a_{1}$,$a_{2}$,$a_{3}$,$a_{4}$,$a_{5}$,$a_{6}$,$a_{7}$,$a_{8}$,$a_{9}$,$a_{10}$,$a_{11}$
$a_{1}$,0.0,0.202,0.257,0.404,0.500,0.500,0.978,0.791,0.981,0.798,0.797
$a_{2}$,0.0,0.000,0.000,0.202,0.000,0.489,0.798,0.595,0.798,0.798,0.595
$a_{3}$,0.0,0.145,0.000,0.113,0.065,0.500,0.798,0.595,0.798,0.798,0.645
$a_{4}$,0.0,0.088,0.000,0.000,0.310,0.500,0.798,0.595,0.798,0.798,0.600
$a_{5}$,0.0,0.045,0.000,0.202,0.000,0.201,0.798,0.595,0.500,0.798,0.595
$a_{6}$,0.0,0.000,0.061,0.202,0.000,0.000,0.798,0.595,0.500,0.798,0.595
$a_{7}$,0.0,0.181,0.027,0.086,0.128,0.202,0.000,0.320,0.000,0.000,0.113
$a_{8}$,0.0,0.149,0.000,0.205,0.070,0.202,0.202,0.000,0.202,0.202,0.054
$a_{9}$,0.0,0.178,0.021,0.078,0.121,0.202,0.298,0.613,0.000,0.595,0.701
$a_{10}$,0.0,0.202,0.196,0.201,0.202,0.202,0.181,0.493,0.183,0.000,0.202


Порахуємо вхідний та вихідний потоки, визначимо чистий потік:

In [10]:
n = agg_pcm.shape[0] - 1
phi_in = agg_pcm.sum(axis=1) / n
phi_out = agg_pcm.sum(axis=0) / n
phi = phi_in - phi_out
display(pd.DataFrame(vstack((phi_in[newaxis, :], phi_out[newaxis, :], phi[newaxis, :])).round(3), columns=alternatives,
        index=['$\Phi^+$','$\Phi^-$','$\Phi$']))

,$a_{1}$,$a_{2}$,$a_{3}$,$a_{4}$,$a_{5}$,$a_{6}$,$a_{7}$,$a_{8}$,$a_{9}$,$a_{10}$,$a_{11}$
$\Phi^+$,0.621,0.427,0.446,0.449,0.373,0.355,0.106,0.129,0.281,0.206,0.135
$\Phi^-$,0.000,0.125,0.056,0.188,0.140,0.320,0.585,0.549,0.496,0.579,0.490
$\Phi$,0.621,0.302,0.389,0.261,0.234,0.035,-0.479,-0.420,-0.215,-0.372,-0.355


Визначимо порядок ранжування альтернатив:

In [11]:
sorted_phi = sort(fromiter(enumerate(phi), dtype=[('index',int),('value',float64)]), order='value')
sorted_alter = ['a_{%d}' % (i + 1) for i in sorted_phi['index']]
order_report = ' \prec '.join(sorted_alter)
order_report = 'Порядок альтернатив за зростанням рангу: $' + order_report + '$.'
display(Latex(order_report))
display(pd.DataFrame(sorted_phi['value'][newaxis,:].round(3), columns=('$' + alt + '$' for alt in sorted_alter), index=['$\Phi$']))

<IPython.core.display.Latex object>

,$a_{7}$,$a_{8}$,$a_{10}$,$a_{11}$,$a_{9}$,$a_{6}$,$a_{5}$,$a_{4}$,$a_{2}$,$a_{3}$,$a_{1}$
$\Phi$,-0.479,-0.42,-0.372,-0.355,-0.215,0.035,0.234,0.261,0.302,0.389,0.621


## Висновок

Під час роботи було виконано ранжування альтернатив за заданими оцінками по критеріям. Для цього було використано метод PROMETHEE II, який полягає у визначенні чистого потоку, та порівняння альтернатив за його значенням.
Один з критеріїв позначав витрати, тому він був інвертований, щоб працювати з ним аналогічно іншим критеріям.
В результаті отримали наступний порядок альтернатив:
$$a_{7} \prec a_{8} \prec a_{10} \prec a_{11} \prec a_{9} \prec a_{6} \prec a_{5} \prec a_{4} \prec a_{2} \prec a_{3} \prec a_{1}.$$

Найкращою виявилася перша альтернатива, найгіршою – сьома.